In [ ]:
-- Explore tables
SELECT COUNT(*) FROM purchases;  -- ~200K records
SELECT COUNT(*) FROM sales;      -- 10M+ records

-- Create optimized vendor summary (joins 4 tables)
CREATE TABLE vendor_sales_summary AS
WITH freight_summary AS (
    SELECT vendor_number, SUM(freight) as freight_cost 
    FROM vendor_invoice GROUP BY vendor_number
),
purchase_summary AS (
    SELECT p.vendor_number, p.brand, 
           pp.price as actual_price, p.purchase_price,
           SUM(p.quantity) as total_purchase_qty,
           SUM(p.dollars) as total_purchase_dollars
    FROM purchases p
    LEFT JOIN purchase_prices pp ON p.brand = pp.brand
    WHERE p.purchase_price > 0
    GROUP BY p.vendor_number, p.brand, p.purchase_price, pp.price
),
sales_summary AS (
    SELECT vendor_number, brand,
           SUM(sales_quantity) as total_sales_qty,
           SUM(sales_dollars) as total_sales_dollars,
           AVG(sales_price) as avg_sales_price
    FROM sales GROUP BY vendor_number, brand
)
SELECT ps.*, ss.total_sales_qty, ss.total_sales_dollars, ss.avg_sales_price,
       fs.freight_cost
FROM purchase_summary ps
LEFT JOIN sales_summary ss ON ps.vendor_number = ss.vendor_number AND ps.brand = ss.brand
LEFT JOIN freight_summary fs ON ps.vendor_number = fs.vendor_number;
